Using the Paper below:
    - "Approximating Mutual Information by Maximum Likelihood Density Ratio Estimation", T. Suzuki et al. 2008
       http://proceedings.mlr.press/v4/suzuki08a/suzuki08a.pdf
       
  https://qiita.com/wsuzume/items/09a59036c8944fd563ff#%E3%82%AB%E3%83%BC%E3%83%8D%E3%83%AB%E5%9B%9E%E5%B8%B0

# カーネル法を用いた相互情報量推定

カーネル法を用いて密度比$r(x, y)$の推定を行い，サンプルから変数$X, Y$の相互情報量の推定値$\hat{MI}(X, Y)$を求める．

### 相互情報量

$$
\begin{align}
    MI(X, Y) &:= \int_{X} \int_{Y} p(x, y) \log \frac{p(x, y)}{p(x)p(y)} ~ dydx \\
    &= \mathbb{E}_{XY} \left[ \log \frac{p(x, y)}{p(x)p(y)} \right] \\
    &\simeq	 \frac{1}{n} \sum_{i=1}^{n} \log \frac{p(x_i, y_i)}{p(x_i)p(y_i)} ~~~ (n \to \infty) \\
\end{align}
$$

ここで，密度比を

$$
    r(x, y) := \frac{p(x, y)}{p(x)p(y)}
$$

とおけば，

$$
    MI(X, Y) \simeq	 \frac{1}{n} \sum_{i=1}^{n} \log r(x, y) ~~~ (n \to \infty) \\
$$



### 密度比 $r(x, y)$ の推定

#### 線形基底回帰

基底関数 $\phi: X^n \times Y^n \to \mathbb{R}^{d}$ と線形重み$\boldsymbol{\beta} \in \mathbb{R}^{d}$を用いて，密度比$r(x, y)$を近似する．

$$
\begin{align}
    r(x, y) &\simeq \hat{r}(x, y) \\
    &= \boldsymbol{\beta}^{\mathrm T} \phi(x, y)  \\
    &= \sum_{i=1}^{d} \beta_i \cdot \phi_i(x, y)
\end{align}
$$

#### カーネル回帰

これはカーネル法を使うと以下のようになる．

$$
\begin{align}
    r(x, y) &\simeq \hat{r}(x, y) \\
    &= \sum_{i=1}^{n} \alpha_i \cdot k \left( {({x}^{(i)}, {y}^{(i)})}^{\mathrm T}, {(x, y)}^{\mathrm T} \right)
\end{align}
$$

たとえば，$k$として，RBFカーネルを用いれば

$$
\begin{align}
    k_{RBF}(x, x') 
    &= \exp \left( - \frac{{|| x - x' || }^2}{2 \sigma^2} \right) = \exp \left( - \gamma {|| x - x' || }^2 \right)
\end{align}
$$

代入して，

$$
\begin{align}
    r(x, y) &\simeq \hat{r}(x, y) \\
    &= \sum_{i=1}^{n} \alpha_i \cdot k \left( {({x}^{(i)}, {y}^{(i)})}^{\mathrm T}, {(x, y)}^{\mathrm T} \right) \\
    &= \sum_{i=1}^{n} \alpha_i \cdot \exp \left( - \gamma { || \left( \begin{array}{c} x^{(i)} \\ y^{(i)} \end{array} \right) - \left( \begin{array}{c} x \\ y \end{array} \right) || }^2 \right)
\end{align}
$$

ハイパーパラメータ$\alpha$の探索のために勾配法を用いる



FInally, we get the estimator $\hat{MI}(X, Y)$ as follows. Note that $\alpha$ and $\gamma$ is the hyperparameters.

$$
\begin{align}
    \hat{MI}(X, Y) &= \frac{1}{n} \sum_{i=1}^{n} \log \hat{r}(x_i y_i) \\ 
    &= \frac{1}{n} \sum_{i=1}^{n} \log \sum_{i=1}^{n} \alpha_i \cdot \exp \left( - \gamma { || \left( \begin{array}{c} x^{(i)} \\ y^{(i)} \end{array} \right) - \left( \begin{array}{c} x \\ y \end{array} \right) || }^2 \right)
\end{align}
$$

DNNの第$i$層:$L_i$および第$j$層:$L_j$の間の相互情報量は，$N$コのサンプル$\{ (x^{(n)}, y^{(n)}) \}_{n=1 \sim N}$から以下の式で推定できる


$$
\begin{align}
    \hat{MI}(L_i, L_j) 
    &= \frac{1}{N} \sum_{n=1}^{N} \log \hat{r}({l_i}^{(n)}, {l_j}^{(n)}) \\ 
    &= \frac{1}{N} \sum_{n=1}^{N} \log \sum_{n=1}^{N} \alpha_n \cdot \exp \left( - \gamma { || \left( \begin{array}{c} {l_i}^{(n)} \\ {l_j}^{(n)} \end{array} \right) - \left( \begin{array}{c} l_i \\ l_j \end{array} \right) || }^2 \right)
\end{align}
$$

## Algorithm for optimize alpha

ある変数$X, Y$に対して，$n$コのサンプル$\{ (x^{(i)}, y^{(i)}) \}_{i=1 \sim n}$から，相互情報量$MI(X, Y)$の推定量を求める．

1. initialize $\boldsymbol{\alpha} = {(\alpha_1, \dots, \alpha_n)}^{\mathrm T}$. i.e. assign mean value for all. $\alpha_i = \frac{1}{n}$

2. foreach step $t$ <br>
    2.1. calculate the log-likelihood $\hat{p}$ of sample $\{ (x^{(i)}, y^{(i)}) \}_{i=1 \sim n}$
    $$
            \alpha_{opt} = \underset{\alpha}{\rm argmax} \sum_{i=1}^{n} \log \sum_{j=1}^{n} \alpha_j \cdot k \left( {({x}^{(i)}, {y}^{(i)})}^{\mathrm T}, {(x^{(j)}, y^{(j)})}^{\mathrm T} \right)
            $$
        

In [56]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [55]:
# RBFカーネル
class GaussianKernel(object):
    
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def get_sigma(self):
        return np.copy(self.sigma)

    def __call__(self, x):
        return np.exp( (-1. / (self.sigma ** 2) ) *  (x - self.mu) ** 2)

    # シグマによる，RBFカーネルの偏導関数
    def derivatives(self, x1, x2):
        dif_sigma = np.exp( (-1. / (self.sigma ** 2) ) *  (x1 - x2) ** 2) * ( (x1 - x2) ** 2 ) / ( self.sigma ** 3)
        return dif_sigma

    # シグマに差分を足しこむ．
    def update_sigma(self, update):
        self.sigma += update
        

In [ ]:
class DensityRatioEstimater():
    
    def __init__(self, x_sample, y_sample, b=200, K=50)
        """
        x_sample: data samples of the variable X
        y_sample: data samples of the variable Y
        z_sample: combine of x_sample & y_sample
        kernel: kernel function (Gaussian kernel is typical)
        b: number of basis function
        K: number of subset of samples. used for CrossValidation.
        """
        
        self.x_sample = x_sample
        self.y_sample = y_sample
        self.z_sample = np.append(x_sample, y_sample)
        self.b = b
        self.K = K
        self.sample_size = len(x_sample)
        
        
    def CrossValidation(self):
        basis_functions = self._make_basisfunctions()
        
        
    def _make_basisfunctions(self):
        # b個のbasis function (GaussianKernel)を作成する．
        # choose the data sample randomly from z_sample
        
        # seed set
        np.random.seed(1)
        
        basis_functions = []
        for i in range(self.b):
            rand_id = np.random.randint(0, self.sample_size)
            basis = GaussianKernel(mu=self.z_sample[rand_id], sigma=2.)
            bassi_functions.append(kernel)
        return basis_functions
        

In [46]:
200 * np.random.rand(1)

array([31.04229391])

In [ ]:
class GaussianProcessRegression(object):

    def __init__(self, kernel, beta=1.):
        self.kernel = kernel
        self.beta = beta

    def fit(self, x, t):
        self.x = x
        self.t = t
        Gram = self.kernel(*np.meshgrid(x, x))
        self.covariance = Gram + np.identity(len(x)) / self.beta
        self.precision = np.linalg.inv(self.covariance)

    def fit_kernel(self, x, t, learning_rate=0.1, iter_max=10000):
        for i in range(iter_max):
            params = self.kernel.get_params()
            self.fit(x, t)
            gradients = self.kernel.derivatives(*np.meshgrid(x, x))
            updates = np.array(
                [-np.trace(self.precision.dot(grad)) + t.dot(self.precision.dot(grad).dot(self.precision).dot(t)) for grad in gradients])
            self.kernel.update_parameters(learning_rate * updates)
            if np.allclose(params, self.kernel.get_params()):
                break
        else:
            print("parameters may not have converged")

    def predict_dist(self, x):
        K = self.kernel(*np.meshgrid(x, self.x, indexing='ij'))
        mean = K.dot(self.precision).dot(self.t)
        var = self.kernel(x, x) + 1 / self.beta - np.sum(K.dot(self.precision) * K, axis=1)
        return mean.ravel(), np.sqrt(var.ravel())


def create_toy_data(func, low=0, high=1., n=10, std=1.):
    x = np.random.uniform(low, high, n)
    t = func(x) + np.random.normal(scale=std, size=n)
    return x, t

In [ ]:
def main():
    def func(x):
        return np.sin(2 * np.pi * x)

    x, t = create_toy_data(func, high=0.7, std=0.1)

    kernel = GaussianKernel(params=np.array([1., 1.]))
    regression = GaussianProcessRegression(kernel=kernel, beta=100)
    regression.fit_kernel(x, t, learning_rate=0.1, iter_max=10000)

    x_test = np.linspace(0, 1, 100)
    y, y_std = regression.predict_dist(x_test)

    plt.scatter(x, t, alpha=0.5, color="blue", label="observation")
    plt.plot(x_test, func(x_test), color="blue", label="sin$(2\pi x)$")
    plt.plot(x_test, y, color="red", label="predict_mean")
    plt.fill_between(x_test, y - y_std, y + y_std, color="pink", alpha=0.5, label="predict_std")
    plt.legend(loc="lower left")
    plt.show()

if __name__ == '__main__':
    main()